# Experiment 002: TabPFN Regressor

TabPFN is a transformer-based foundation model specifically designed for small tabular datasets.
Published in Nature 2025, it "consistently beats standard MLPs and gradient-boosted trees on low-sample drug-discovery benchmarks."

Implementing:
- TabPFN Regressor for each target (3 separate models)
- Same Arrhenius kinetics features + Spange descriptors
- TTA for mixed solvent symmetry
- No hyperparameter tuning needed

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from abc import ABC
import tqdm
import warnings
warnings.filterwarnings('ignore')

# Check GPU
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

torch.set_default_dtype(torch.double)

GPU available: True
GPU: NVIDIA H100 80GB HBM3


In [2]:
# Test TabPFN import
from tabpfn import TabPFNRegressor
print("TabPFN imported successfully!")

# Quick test
test_model = TabPFNRegressor()
print(f"TabPFN model created: {type(test_model)}")

TabPFN imported successfully!


TabPFN model created: <class 'tabpfn.regressor.TabPFNRegressor'>


In [3]:
# Data loading utilities
DATA_PATH = '/home/data'

INPUT_LABELS_FULL_SOLVENT = [
    "Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%",
]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
TARGET_LABELS = ["Product 2", "Product 3", "SM"]

def load_data(name="full"):
    assert name in ["full", "single_solvent"]
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[TARGET_LABELS]
    return X, Y

def load_features(name="spange_descriptors"):
    return pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent_name in sorted(X["SOLVENT NAME"].unique()):
        train_mask = X["SOLVENT NAME"] != solvent_name
        yield (X[train_mask], Y[train_mask]), (X[~train_mask], Y[~train_mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates().sort_values(["SOLVENT A NAME", "SOLVENT B NAME"])
    for _, row in ramps.iterrows():
        train_mask = (X[["SOLVENT A NAME", "SOLVENT B NAME"]] != row).any(axis=1)
        yield (X[train_mask], Y[train_mask]), (X[~train_mask], Y[~train_mask])

# Load Spange descriptors
SPANGE_DF = load_features('spange_descriptors')
print(f"Spange descriptors shape: {SPANGE_DF.shape}")

Spange descriptors shape: (26, 13)


In [ ]:
# Featurizer with Arrhenius kinetics
class KineticFeaturizer:
    """Featurizer with Arrhenius kinetics features."""
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.featurizer = SPANGE_DF
        self.feats_dim = self.featurizer.shape[1] + 2 + 3  # 18 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        
        # Arrhenius kinetics features
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        # Solvent features
        if self.mixed:
            A = self.featurizer.loc[X["SOLVENT A NAME"]].values
            B = self.featurizer.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            
            if flip:
                X_chem = B * (1 - (1-pct)) + A * (1-pct)
            else:
                X_chem = A * (1 - pct) + B * pct
        else:
            X_chem = self.featurizer.loc[X["SOLVENT NAME"]].values
            
        return np.hstack([X_kinetic, X_chem])

In [ ]:
# TabPFN Model with TTA support
class TabPFNModel:
    def __init__(self, data='single', use_tta=True):
        self.data_type = data
        self.use_tta = use_tta and (data == 'full')  # TTA only for mixed solvents
        self.featurizer = KineticFeaturizer(mixed=(data=='full'))
        self.models = []  # One per target
        
    def train_model(self, X_train, y_train):
        X_feats = self.featurizer.featurize(X_train, flip=False)
        
        if self.data_type == 'full' and self.use_tta:
            # Data augmentation with symmetric solvent swapping
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = np.vstack([X_feats, X_flip])
            y_all = np.vstack([y_train.values, y_train.values])
        else:
            X_all = X_feats
            y_all = y_train.values
        
        self.models = []
        for i in range(3):  # 3 targets
            model = TabPFNRegressor()
            model.fit(X_all, y_all[:, i])
            self.models.append(model)
    
    def predict(self, X_test):
        X_feats = self.featurizer.featurize(X_test, flip=False)
        
        if self.use_tta:
            # TTA: predict with both orderings and average
            X_flip = self.featurizer.featurize(X_test, flip=True)
            
            preds_std = np.column_stack([m.predict(X_feats) for m in self.models])
            preds_flip = np.column_stack([m.predict(X_flip) for m in self.models])
            preds = (preds_std + preds_flip) / 2
        else:
            preds = np.column_stack([m.predict(X_feats) for m in self.models])
        
        # Clip to [0, 1]
        preds = np.clip(preds, 0, 1)
        return torch.tensor(preds)

In [ ]:
# Test on single fold
print("Testing TabPFN on single fold...")
X, Y = load_data("single_solvent")
print(f"Single solvent data: {X.shape[0]} samples")

split_gen = generate_leave_one_out_splits(X, Y)
(train_X, train_Y), (test_X, test_Y) = next(split_gen)
print(f"Train: {len(train_X)}, Test: {len(test_X)}")

model = TabPFNModel(data='single')
model.train_model(train_X, train_Y)

preds = model.predict(test_X)
print(f"Predictions shape: {preds.shape}")
print(f"Predictions range: [{preds.min():.4f}, {preds.max():.4f}]")

rmse = np.sqrt(((preds.numpy() - test_Y.values) ** 2).mean())
print(f"Single fold RMSE: {rmse:.4f}")

In [ ]:
# Full CV for single solvent task
print("\n" + "="*50)
print("TASK 0: Single Solvent (Leave-One-Solvent-Out CV)")
print("="*50)

X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
fold_rmses = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = TabPFNModel(data='single')
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X).numpy()
    
    fold_rmse = np.sqrt(((predictions - test_Y.values) ** 2).mean())
    fold_rmses.append(fold_rmse)
    
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 0, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })

submission_single_solvent = pd.DataFrame(all_predictions)
print(f"\nSingle Solvent CV RMSE: {np.mean(fold_rmses):.5f} ± {np.std(fold_rmses):.5f}")

In [ ]:
# Full CV for full data task
print("\n" + "="*50)
print("TASK 1: Full Data (Leave-One-Ramp-Out CV)")
print("="*50)

X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []
fold_rmses_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = TabPFNModel(data='full', use_tta=True)
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X).numpy()
    
    fold_rmse = np.sqrt(((predictions - test_Y.values) ** 2).mean())
    fold_rmses_full.append(fold_rmse)
    
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 1, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })

submission_full_data = pd.DataFrame(all_predictions)
print(f"\nFull Data CV RMSE: {np.mean(fold_rmses_full):.5f} ± {np.std(fold_rmses_full):.5f}")

In [ ]:
# Save submission
submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"

submission.to_csv("/home/code/experiments/002_tabpfn/submission.csv", index=True)
submission.to_csv("/home/submission/submission.csv", index=True)

print(f"\nSubmission saved with {len(submission)} rows")

In [ ]:
# Final summary
print("\n" + "="*50)
print("FINAL RESULTS - TabPFN")
print("="*50)
print(f"Single Solvent CV RMSE: {np.mean(fold_rmses):.5f} ± {np.std(fold_rmses):.5f}")
print(f"Full Data CV RMSE: {np.mean(fold_rmses_full):.5f} ± {np.std(fold_rmses_full):.5f}")

overall_rmse = (np.mean(fold_rmses) + np.mean(fold_rmses_full)) / 2
print(f"\nOverall CV RMSE: {overall_rmse:.5f}")

print(f"\nComparison:")
print(f"  Baseline MLP: 0.08819")
print(f"  TabPFN:       {overall_rmse:.5f}")
print(f"  Target:       0.04740")